In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import make_interp_spline

/tmp/ipykernel_2303/3321561094.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
flag_project = 1 # 0--regular ; 1--repartrition

In [3]:
if flag_project == 0:
    file_project = 'a-project.yml'
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.2_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
dir_disp_autopick = dir_project + info_basic['dir_disp']

In [7]:
key_subworks = info_basic['key_subworks']
key_subworks = info_basic['key_subworks_repick']
key_subworks

['394--11-13',
 '62--42-04',
 '484--20-15',
 '483--19-15',
 '45--20-04',
 '422--41-13',
 '77--26-05',
 '500--45-15',
 '583--35-18',
 '351--13-12',
 '48--23-04',
 '533--46-16',
 '49--24-04',
 '434--53-13',
 '126--43-06',
 '506--51-15',
 '113--25-06',
 '518--22-16',
 '46--21-04',
 '248--39-09',
 '278--27-10',
 '363--26-12',
 '472--52-14',
 '98--53-05',
 '481--15-15',
 '586--38-18',
 '93--44-05',
 '392--09-13',
 '135--53-06',
 '128--45-06',
 '95--49-05',
 '14--22-03',
 '530--43-16',
 '130--47-06',
 '528--41-16',
 '166--42-07',
 '165--41-07',
 '501--46-15',
 '529--42-16',
 '527--40-16',
 '292--41-10',
 '433--52-13',
 '72--16-05',
 '291--40-10',
 '123--39-06',
 '375--39-12',
 '181--13-08',
 '504--49-15',
 '47--22-04',
 '74--19-05',
 '378--42-12',
 '97--52-05',
 '379--43-12',
 '448--24-14',
 '462--42-14',
 '556--35-17',
 '131--48-06',
 '563--46-17',
 '451--27-14',
 '440--13-14',
 '134--51-06',
 '129--46-06',
 '175--51-07',
 '203--38-08',
 '503--48-15',
 '125--42-06',
 '580--18-18',
 '507--52

### create workspace for Dispernet

In [8]:
info_basic['dir_inv_dispernet'] = 'inv_dispernet/'
dir_inv = dir_project + info_basic['dir_inv_dispernet']
if not os.path.exists(dir_inv):
    os.makedirs(dir_inv)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [9]:
dir_disp = dir_inv + 'data/'
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)

### Autopick

In [10]:
flag_f = [5,13]

In [11]:
def transfer_autopick(key_subwork):
    global dir_disp_autopick, dir_disp, flag_f
    print(key_subwork)
    ff_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_start = flag_f[0]
    f_end = flag_f[1]
    ff_this = ff_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    cc_this = cc_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    mode = np.zeros(len(ff_this))
    np.savetxt(dir_disp+'ds_'+key_subwork+'curve.txt',np.c_[ff_this,cc_this,mode], fmt='%.6f  %.6f  %i')

In [12]:
for key_subwork in key_subworks:
    #print(key_subwork)
    transfer_autopick(key_subwork)

394--11-13


62--42-04
484--20-15
483--19-15
45--20-04
422--41-13
77--26-05
500--45-15
583--35-18
351--13-12
48--23-04
533--46-16
49--24-04
434--53-13
126--43-06
506--51-15
113--25-06
518--22-16
46--21-04
248--39-09
278--27-10
363--26-12
472--52-14
98--53-05
481--15-15
586--38-18
93--44-05
392--09-13
135--53-06
128--45-06
95--49-05
14--22-03
530--43-16
130--47-06
528--41-16
166--42-07
165--41-07
501--46-15
529--42-16
527--40-16
292--41-10
433--52-13
72--16-05
291--40-10
123--39-06
375--39-12
181--13-08
504--49-15
47--22-04
74--19-05
378--42-12
97--52-05
379--43-12
448--24-14
462--42-14
556--35-17
131--48-06
563--46-17
451--27-14
440--13-14
134--51-06
129--46-06
175--51-07
203--38-08
503--48-15
125--42-06
580--18-18
507--52-15
389--53-12
133--50-06
277--26-10
458--38-14
432--51-13
474--54-14
380--44-12
431--50-13
13--21-03
482--16-15
424--43-13
480--14-15
456--36-14
558--37-17
559--38-17
557--36-17
179--10-08
376--40-12
584--36-18
180--12-08
334--42-11
459--39-14
114--26-06
247--38-09
382--46-12
50-